# Machine Learning Foundation

## Section 2, Part e: Regularization LAB 

## Learning objectives

By the end of this lesson, you will be able to:

* Implement data standardization
* Implement variants of regularized regression
* Combine data standardization with the train-test split procedure
* Implement regularization to prevent overfitting in regression problems

In [11]:
import numpy as np
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_42464b81c6f9448e8715d6fd31794fae = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6qlO7DtMy-0aFKD8lHsi_S2XcyD0G5gOiBHWbV5-QW0M',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

body = client_42464b81c6f9448e8715d6fd31794fae.get_object(Bucket='ibmml12-donotdelete-pr-xvar8fhwxguq8w',Key='ibm_ml1_2e.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

boston_data = pd.read_csv(body)
boston_data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


### Loading in Boston Data

**Note:** See `helper.py` file to see how boston data is read in from SciKit Learn.

## Data standardization

**Standardizing** data refers to transforming each variable so that it more closely follows a **standard** normal distribution, with mean 0 and standard deviation 1.

The [`StandardScaler`](http://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) object in SciKit Learn can do this.

**Generate X and y**:

In [13]:
y_col = "medv"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

**Import, fit, and transform using `StandardScaler`**

In [14]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()
X_ss = s.fit_transform(X)

### Exercise: 

Confirm standard scaling

In [18]:
X_ss.mean(axis=0)
X_ss.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Hint:

### Coefficients with and without scaling

In [19]:
from sklearn.linear_model import LinearRegression

In [20]:
lr = LinearRegression()

y_col = "medv"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

In [21]:
lr.fit(X, y)
print(lr.coef_) # min = -18

[-1.08011358e-01  4.64204584e-02  2.05586264e-02  2.68673382e+00
 -1.77666112e+01  3.80986521e+00  6.92224640e-04 -1.47556685e+00
  3.06049479e-01 -1.23345939e-02 -9.52747232e-01  9.31168327e-03
 -5.24758378e-01]


#### Discussion (together): 

The coefficients are on widely different scales. Is this "bad"?

In [22]:
from sklearn.preprocessing import StandardScaler

In [23]:
s = StandardScaler()
X_ss = s.fit_transform(X)

In [24]:
lr2 = LinearRegression()
lr2.fit(X_ss, y)
print(lr2.coef_) # coefficients now "on the same scale"

[-0.92814606  1.08156863  0.1409      0.68173972 -2.05671827  2.67423017
  0.01946607 -3.10404426  2.66221764 -2.07678168 -2.06060666  0.84926842
 -3.74362713]


In [26]:
X.columns

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat'],
      dtype='object')

### Exercise: 

Based on these results, what is the most "impactful" feature (this is intended to be slightly ambiguous)? "In what direction" does it affect "y"?

**Hint:** Recall from last week that we can "zip up" the names of the features of a DataFrame `df` with a model `model` fitted on that DataFrame using:

```python
dict(zip(df.columns.values, model.coef_))
```

In [27]:
### BEGIN SOLUTION
pd.DataFrame(zip(X.columns, lr2.coef_)).sort_values(by=1)
### END SOLUTION

,0,1
12,lstat,-3.743627
7,dis,-3.104044
9,tax,-2.076782
10,ptratio,-2.060607
4,nox,-2.056718
0,crim,-0.928146
6,age,0.019466
2,indus,0.140900
3,chas,0.681740
11,b,0.849268


Looking just at the strength of the standardized coefficients LSTAT, DIS, RM and RAD are all the 'most impactful'. Sklearn does not have built in statistical signifigance of each of these variables which would aid in making this claim stronger/weaker

### Lasso with and without scaling

We discussed Lasso in lecture. 

Let's review together:

1. What is different about Lasso vs. regular Linear Regression?
1. Is standardization more or less important with Lasso vs. Linear Regression? Why?

In [28]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures

#### Create polynomial features

[`PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

In [29]:
pf = PolynomialFeatures(degree=2, include_bias=False)
X_pf = pf.fit_transform(X)

**Note:** We use `include_bias=False` since `Lasso` includes a bias by default.

In [30]:
X_pf_ss = s.fit_transform(X_pf)

### Lasso

[`Lasso` documentation](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

In [31]:
las = Lasso()
las.fit(X_pf_ss, y)
las.coef_ 

array([-0.        ,  0.        , -0.        ,  0.        , -0.        ,
        0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.99073601,  0.        , -0.        , -0.        ,  0.        ,
       -0.        ,  0.06751978, -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.05012457, -0.        ,
       -0.        , -0.        , -0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        , -0.        ,  3.30027759, -0.  

### Exercise

Compare 

* Sum of magnitudes of the coefficients
* Number of coefficients that are zero

for Lasso with alpha 0.1 vs. 1.

Before doing the exercise, answer the following questions in one sentence each:

* Which do you expect to have greater magnitude?
* Which do you expect to have more zeros?

In [38]:
las2 = Lasso(alpha=0.1)
las2.fit(X_pf_ss, y)
print('sum of abs values is ',(np.abs(las2.coef_)).sum(), 'num of zeros is ', (las2.coef_==0).sum())
print('old sum of abs values is ',(np.abs(las.coef_)).sum(), 'old num of zeros is ', (las.coef_==0).sum())

sum of abs values is  26.138682362877965 num of zeros is  81
old sum of abs values is  8.472405044553073 old num of zeros is  97


With more regularization (higher alpha) we will expect the penalty for higher weights to be greater and thus the coefficients to be pushed down. Thus a higher alpha means lower magnitude with more coefficients pushed down to 0.

### Exercise: $R^2$

Calculate the $R^2$ of each model without train/test split.

Recall that we import $R^2$ using:

```python
from sklearn.metrics import r2_score
```

In [40]:
### BEGIN SOLUTION
from sklearn.metrics import r2_score
print(r2_score(y,las.predict(X_pf_ss)))
print(r2_score(y,las2.predict(X_pf_ss)))
### END SOLUTION

0.7207000417838496
0.8366045990855128


#### Discuss:

Will regularization ever increase model performance if we evaluate on the same dataset that we trained on?

## With train/test split

#### Discuss

Are there any issues with what we've done so far?

**Hint:** Think about the way we have done feature scaling.

Discuss in groups of two or three.

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, 
                                                    random_state=72018)

In [43]:
X_train_s = s.fit_transform(X_train)
las.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las.predict(X_test_s)
r2_score(y_pred, y_test)

0.33177406838134427

In [45]:
X_train_s = s.fit_transform(X_train)
las2.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred = las2.predict(X_test_s)
r2_score(y_pred, y_test)

0.7102444090776477

### Exercise


#### Part 1:

Do the same thing with Lasso of:

* `alpha` of 0.001
* Increase `max_iter` to 100000 to ensure convergence. 

Calculate the $R^2$ of the model.

Feel free to copy-paste code from above, but write a one sentence comment above each line of code explaining why you're doing what you're doing.

#### Part 2:

Do the same procedure as before, but with Linear Regression.

Calculate the $R^2$ of this model.

#### Part 3: 

Compare the sums of the absolute values of the coefficients for both models, as well as the number of coefficients that are zero. Based on these measures, which model is a "simpler" description of the relationship between the features and the target?

In [46]:
### BEGIN SOLUTION

# Part 1

# Decreasing regularization and ensuring convergence
las001 = Lasso(alpha = 0.001, max_iter=100000)

# Transforming training set to get standardized units
X_train_s = s.fit_transform(X_train)

# Fitting model to training set
las001.fit(X_train_s, y_train)

# Transforming test set using the parameters defined from training set
X_test_s = s.transform(X_test)

# Finding prediction on test set
y_pred = las001.predict(X_test_s)

# Calculating r2 score
print("r2 score for alpha = 0.001:", r2_score(y_pred, y_test))


# Part 2

# Using vanilla Linear Regression
lr = LinearRegression()

# Fitting model to training set
lr.fit(X_train_s, y_train)

# predicting on test set
y_pred_lr = lr.predict(X_test_s)

# Calculating r2 score
print("r2 score for Linear Regression:", r2_score(y_pred_lr, y_test))


# Part 3
print('Magnitude of Lasso coefficients:', abs(las001.coef_).sum())
print('Number of coeffients not equal to 0 for Lasso:', (las001.coef_!=0).sum())

print('Magnitude of Linear Regression coefficients:', abs(lr.coef_).sum())
print('Number of coeffients not equal to 0 for Linear Regression:', (lr.coef_!=0).sum())
### END SOLUTION

r2 score for alpha = 0.001: 0.8686454101886679
r2 score for Linear Regression: 0.8555202098064155
Magnitude of Lasso coefficients: 436.26164263066767
Number of coeffients not equal to 0 for Lasso: 89
Magnitude of Linear Regression coefficients: 1185.2858254467615
Number of coeffients not equal to 0 for Linear Regression: 104


## L1 vs. L2 Regularization

As mentioned in the deck: `Lasso` and `Ridge` regression have the same syntax in SciKit Learn.

Now we're going to compare the results from Ridge vs. Lasso regression:

[`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

In [47]:
from sklearn.linear_model import Ridge

### Exercise

Following the Ridge documentation from above:

1. Define a Ridge object `r` with the same `alpha` as `las001`.
2. Fit that object on `X` and `y` and print out the resulting coefficients.

In [48]:
### BEGIN SOLUTION
# Decreasing regularization and ensuring convergence
r = Ridge(alpha = 0.001)
X_train_s = s.fit_transform(X_train)
r.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_r = r.predict(X_test_s)

# Calculating r2 score
r.coef_
### END SOLUTION

array([  8.72689888,  10.35438847, -24.68294626,   5.29914719,
        -3.13016302,  14.78105728,  22.30136264, -23.48305783,
        27.35606088,  -1.51226982,  16.94932767,  22.5310337 ,
        11.44446396,   1.055279  ,   0.43731614,  14.25939284,
         1.83002371,  -8.73650547,   4.72642374,  -3.74840394,
        -3.55691683, -16.52096879,  -6.91839919,   6.06377957,
        -1.36292977,   4.60368923,  -1.30313233,  -0.05700059,
        -0.30136319, -12.82750449,   2.0162355 ,   1.08432655,
        -0.65484801,  -1.09243931,   4.4575366 ,  -4.24739604,
         4.15606087,  -1.28186218,   8.7098614 ,  -0.28471929,
        -8.59503823,  11.59516377,   6.54406943,   1.27107172,
         1.82652783,   1.0073212 ,   1.55365183,   5.17299733,
        -4.68495093,   5.29914719,  -3.19877584,  -8.71503739,
         0.9828974 ,   1.11942935,   0.28373611,  -1.6832555 ,
        -2.88397099,   2.98542984,  -0.7557536 ,  12.29670631,
         0.93050194,  -7.59356615,  18.29163699, -22.12

In [49]:
las001 # same alpha as Ridge above

Lasso(alpha=0.001, max_iter=100000)

In [50]:
las001.coef_

array([  0.        ,   0.        , -16.94519717,   2.56249651,
         0.        ,  13.39183589,   9.90704376, -19.9595387 ,
         9.03929536,   0.        ,   6.13848021,  17.62797548,
        11.44584146,   1.2103543 ,   0.22608228,  11.57792302,
         2.19749709,  -7.3973882 ,   4.11704646,  -1.58851015,
        -2.15697213,  -9.83681606,   0.        ,   0.        ,
        -1.0887272 ,   3.79250131,   0.38919774,   0.19975916,
        -0.29717348,  -3.35192712,   0.39571905,   0.71942421,
        -0.75600102,  -0.75333552,   2.3883691 ,  -0.84488476,
         2.52363513,  -0.97275961,   3.88249629,  -0.98265027,
         6.60948206,   6.13776516,   4.20054331,   0.85845288,
         1.96027249,   2.66131873,  -4.14837832,   2.93685228,
        -4.55591102,   2.11214343,  -1.98303346,  -7.4388542 ,
         1.60663589,   1.66843473,  -1.2622367 ,  -0.        ,
        -0.        ,   2.62178221,  -0.86086897,   3.35340034,
        -0.        ,  -2.45094401,   9.17065688,  -6.91

In [51]:
print(np.sum(np.abs(r.coef_)))
print(np.sum(np.abs(las001.coef_)))

print(np.sum(r.coef_ != 0))
print(np.sum(las001.coef_ != 0))

795.6521694369505
436.26164263066767
104
89


**Conclusion:** Ridge does not make any coefficients 0. In addition, on this particular dataset, Lasso provides stronger overall regularization than Ridge for this value of `alpha` (not necessarily true in general).

In [52]:
y_pred = r.predict(X_pf_ss)
print(r2_score(y, y_pred))

y_pred = las001.predict(X_pf_ss)
print(r2_score(y, y_pred))

0.9075278340575201
0.9102933722688219


**Conclusion**: Ignoring issues of overfitting, Ridge does slightly better than Lasso when `alpha` is set to 0.001 for each (not necessarily true in general).

# Example: Does it matter when you scale?

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_ss, y, test_size=0.3, 
                                                    random_state=72018)

In [54]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_pred, y_test)

0.5832942094971795

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=72018)

In [56]:
s = StandardScaler()
lr_s = LinearRegression()
X_train_s = s.fit_transform(X_train)
lr_s.fit(X_train_s, y_train)
X_test_s = s.transform(X_test)
y_pred_s = lr_s.predict(X_test_s)
r2_score(y_pred_s, y_test)

0.5832942094971799

**Conclusion:** It doesn't matter whether you scale before or afterwards, in terms of the raw predictions, for Linear Regression. However, it matters for other algorithms. Plus, as we'll see later, we can make scaling part of a `Pipeline`.

---
### Machine Learning Foundation (C) 2020 IBM Corporation